In [ ]:
import pandas as pd
import numpy as np
import json
import requests
from pandas.io.json import json_normalize
import time
import glob as glob
import threading
import matplotlib.pyplot as plt

%matplotlib inline

### Twitch API

Objective: 
- leverage Twitch's API to request 2019-2020 data. 

Purpose: 
- Twitch is a market leader in sVoD. I want to include them in my "Streaming Amid Pandemic" analysis.

Endpoints: 
- user subscriptions
- top streams
- time watched
- top titles watched by week
- top 10 genres
- top 10 titles by genre

Steps: 
- register for api 
- authenticate 
- access
- call API for endpoints above
- don't forget function
- execute initial eda in python
- ship csv for tidy eda
- mvp the twitch tableau dashboard

Documentation:
- https://dev.twitch.tv/docs/
- https://oauth.net/2/

In [ ]:
#testing api

top_games_df='https://api.twitch.tv/helix/games/top'
result=requests.get(top_games_df)
result.status_code

In [ ]:
""" GET https://id.twitch.tv/oauth2/authorize
    ?client_id='***'
    &redirect_uri='*****'
    &response_type=client_id
    &scope=<space-separated list of scopes>
    
    

In [ ]:
print(access_code)

In [ ]:
access_code.text

In [ ]:
#access token

token = json.loads(access_code.text)
access_token = token['access_token']

'''

sample response:

{
  "access_token": "****",
  "refresh_token": "",
  "expires_in": 3600,
  "scope": [],
  "token_type": "bearer"
}

'''

In [ ]:
'''

#curl -H 'Client-ID: *****' \
#-H 'Authorization: Bearer *****' \
#-X GET 'https://api.twitch.tv/helix/games/top'

This might be the wrong direction. I think it's more for users you want to authenticate to your registered redirect URI.

'''

In [ ]:
'''

POST https://id.twitch.tv/oauth2/token
    ?client_id='***'
    &client_secret='****'
    &grant_type=client_credentials
    &scope='top-games'

'''

#OAuth issuse...come back to this after class is out to fix api, use csvs for now

### Pivoting to csvs 
- timeframe: 2016-2020
- compile into dfs for games and channels
- extent: top 100

In [ ]:
most_streamed_df = pd.read_csv('../data/twitch_most_streamed.csv')
most_streamed_df.shape

In [ ]:
most_streamed_df

In [ ]:
#dropping unnamed 
most_streamed_df = most_streamed_df.loc[:, ~most_streamed_df.columns.str.contains('^Unnamed')]
most_streamed_df.head()

In [ ]:
most_watched_df = pd.read_csv('../data/twitch_most_watched.csv')
most_watched_df.shape

In [ ]:
most_watched_df.head()

In [ ]:
#dropping unnamed 
most_watched_df = most_watched_df.loc[:, ~most_watched_df.columns.str.contains('^Unnamed')]
most_watched_df.head(1)

In [ ]:
largest_viewership_df = pd.read_csv('../data/twitch_largest_peak_viewership_year.csv')
largest_viewership_df.shape

In [ ]:
largest_viewership_df.head()

In [ ]:
#dropping unnamed 
largest_viewership_df = largest_viewership_df.loc[:, ~largest_viewership_df.columns.str.contains('^Unnamed')]
largest_viewership_df.head(1)

In [ ]:
peak_viewership_df = pd.read_csv('../data/twitch_peaks.csv')
peak_viewership_df.shape

In [ ]:
peak_viewership_df.head()

In [ ]:
#dropping unnamed 
peak_viewership_df = peak_viewership_df.loc[:, ~peak_viewership_df.columns.str.contains('^Unnamed')]
peak_viewership_df.head(1)

Note: Just use the frist csv. Don't need bottom three to do further analysis, python can handle it via first one. Must have exported redunant csvs or renamed wrong upon initial inspection.

Vizulaization idea: Possibly an animation showing viewers engagement over time? Something like... 50 most watched channels by week for q4 2019 through q1 2020. (the first figure here might be a good direction: https://www.tableau.com/about/blog/2020/2/bring-your-data-life-viz-animations). Use bubble size to indicate follower gain and viewership over 7 day rolling averages. y-axis = viewer count average by week. x-axis = followers per month. 

In [ ]:
twitch_concat_df.head()

### Update: scratch that above. bring all dfs together for channels and games. Could probably run a generator expression and add date year month columns, ie:

### pd.concat((pd.read_csv(file).assign(filename = stats)
### for stats in stats_2020), ignore_index=True)

In [ ]:
'''

#method 2 
#making a concatination function to multiple csv files into one csv. try this for channels and games folders of months 2016-2020, should be 106 csvs.

def concatenate(indir="/Users/home/Desktop/nss_data_analytics/capstone/streaming_analysis_amid_pandemic/data/narrowed/twitch/stats/2020",outfile="/Users/home/Desktop/nss_data_analytics/capstone/streaming_analysis_amid_pandemic/data/tidy/twitch/concatenated.csv"):
    os.chdir(indir)
    fileList=glob.glob("*.csv")
    dfList=[]
    colnames=["name",
              "rank",
              "game",
              "watch_time",
              "stream_time",
              "peak_viewers",
              "peak_channels",
              "streamers",
              "average_viewers",
              "average_channels",
              "average_viewer_ratio",
              "name"]
    
    #looping
    for filname in fileList:
        print(filname)
        df=pandas.read_csv(filename,header=None)
        dfList.append(df)
    concatDf=pandasdas.concat(dfList,axis=0)
    concatDf.columns=colnames
    concat.to_csv(outfile,index=None)

"""

